In [14]:
#Import libraries
import pandas as pd
import pulp

In [16]:
# Read the input data csv into df
df = pd.read_csv("biourja-efzrr-y7i38ed9-input.csv")

df["Ratio"] = (df["Forecast"] / sum(df["Forecast"].values))

# Given regional/zonal forecast values
east_forecast = 2800
north_forecast = 1500
west_forecast = 2000
south_forecast = 6500

# Given total state forecast values
total_state_forecast = 12000

# Create a LP problem
problem = pulp.LpProblem("My_LP_Problem", pulp.LpMinimize)


# Creating LP variables for E1 to E24 with lower bound as zero and upper bound as Capacity of individual wind farms
east_forecast_var = [pulp.LpVariable(f'E{i}', lowBound=0, upBound=df.loc[df['Plant_Name']== f'E{i}']["Capacity"].values[0], cat=pulp.LpContinuous) for i in range(1, 25)]

# Creating LP variables for N1 to N26 with lower bound as zero and upper bound as Capacity of individual wind farms 
north_forecast_var = [pulp.LpVariable(f'N{i}', lowBound=0, upBound=df.loc[df['Plant_Name']== f'N{i}']["Capacity"].values[0], cat=pulp.LpContinuous) for i in range(1, 27)]

# Creating LP variables for W1 to W15 with lower bound as zero and upper bound as Capacity of individual wind farms
west_forecast_var = [pulp.LpVariable(f'W{i}', lowBound=0, upBound=df.loc[df['Plant_Name']== f'W{i}']["Capacity"].values[0], cat=pulp.LpContinuous) for i in range(1, 16)]

# Creating LP variables for S1 to S35 with lower bound as zero and upper bound as Capacity of individual wind farms
south_forecast_var = [pulp.LpVariable(f'S{i}', lowBound=0, upBound=df.loc[df['Plant_Name']== f'S{i}']["Capacity"].values[0], cat=pulp.LpContinuous) for i in range(1, 36)]

# All zone variables aggregated
variables = east_forecast_var + north_forecast_var + west_forecast_var + south_forecast_var

# Objective Function
problem += pulp.lpSum(east_forecast_var) + pulp.lpSum(north_forecast_var) + pulp.lpSum(west_forecast_var) + pulp.lpSum(south_forecast_var) - total_state_forecast

# Constraints
# Bounding the objective function to zero as a minimum
problem += (pulp.lpSum(east_forecast_var) + pulp.lpSum(north_forecast_var) + pulp.lpSum(west_forecast_var) + pulp.lpSum(south_forecast_var) - total_state_forecast) >= 0

# 1. Wind farms should be redispatched in the ratio of their current individual level forecast compared to their capacity
for var in variables:
    problem += var == ((df.loc[df['Plant_Name']== var.name]["Ratio"].values[0]) * total_state_forecast)

# 2. Ratio of other Regional/zonal level output should match that of  regional/zonal level forecast
problem += pulp.lpSum(east_forecast_var) * north_forecast == pulp.lpSum(north_forecast_var) * east_forecast
problem += pulp.lpSum(east_forecast_var) * west_forecast == pulp.lpSum(west_forecast_var) * east_forecast
problem += pulp.lpSum(south_forecast_var) * west_forecast == pulp.lpSum(west_forecast_var) * south_forecast

# 3rd constraint already covered while initializing LpVariable with upBound

# Solve the linear programming problem
problem.solve()

if pulp.LpStatus[problem.status] == "Optimal":
    print("Optimal solution found.")
else:
    print("No feasible solution found.")


# Printing the final solution 
# print(pulp.value(problem.objective))

for var in variables:
    print(var.name,pulp.value(var))

No feasible solution found.
E1 43.981594
E2 30.225236
E3 50.0
E4 100.0
E5 150.0
E6 300.0
E7 100.0
E8 111.79462
E9 100.0
E10 200.0
E11 200.0
E12 251.39259
E13 100.0
E14 100.0
E15 400.0
E16 100.0
E17 853.08469
E18 50.0
E19 150.0
E20 150.0
E21 50.0
E22 50.0
E23 100.0
E24 150.0
N1 163.23565
N2 0.0
N3 182.12642
N4 0.0
N5 0.0
N6 0.0
N7 220.87673
N8 0.0
N9 169.53257
N10 214.48293
N11 0.0
N12 0.0
N13 0.0
N14 198.9828
N15 0.0
N16 0.0
N17 0.0
N18 85.831921
N19 0.0
N20 0.0
N21 0.0
N22 215.64543
N23 292.08041
N24 0.0
N25 27.512715
N26 313.87745
W1 214.67668
W2 100.0
W3 150.0
W4 150.0
W5 200.0
W6 100.0
W7 100.0
W8 50.0
W9 150.0
W10 100.0
W11 150.0
W12 200.0
W13 50.0
W14 150.0
W15 914.2367
S1 150.0
S2 150.0
S3 350.0
S4 100.0
S5 50.0
S6 350.0
S7 100.0
S8 50.0
S9 50.0
S10 200.0
S11 2489.0127
S12 100.0
S13 50.0
S14 150.0
S15 200.0
S16 250.0
S17 216.42044
S18 282.29596
S19 100.0
S20 100.0
S21 200.0
S22 386.53427
S23 50.0
S24 313.6837
S25 407.55631
S26 100.0
S27 56.962945
S28 350.0
S29 450.0
S30 300.0
S3